In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("drive/My Drive/Colab Notebooks")

In [0]:
import numpy as np
import matplotlib.pyplot as plt
# for creating validation set
from sklearn.model_selection import train_test_split
# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout, ELU
from torch.optim import Adam, SGD

In [0]:
# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [5]:
device

device(type='cuda')

In [0]:
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

In [7]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [8]:
# create labels 
y_train_valid = y_train_valid - 769
y_test = y_test -769

person_train_valid = person_train_valid.squeeze()


# split training and validation
X_train, X_val, y_train, y_val, person_train, person_val = train_test_split(X_train_valid, y_train_valid, person_train_valid, test_size = 0.1)
data = []
(X_train.shape, y_train.shape, person_train.shape), (X_val.shape, y_val.shape, person_val.shape)

(((1903, 22, 1000), (1903,), (1903,)), ((212, 22, 1000), (212,), (212,)))

In [0]:
# transform to torch tensor
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)

X_val = torch.from_numpy(X_val)
y_val = torch.from_numpy(y_val)

X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)


In [0]:
class InceptionNet(nn.Module):
    def __init__(self):
        super(InceptionNet, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=22, out_channels=32, kernel_size=(1,9), stride=(1,1), padding=(0,4))
        self.conv0_bn = nn.BatchNorm2d(32)
        self.conv1 = nn.Conv2d(in_channels=22, out_channels=32, kernel_size=(1,7), stride=(1,1), padding=(0,3))
        self.conv1_bn = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=22, out_channels=32, kernel_size=(1,5), stride=(1,1), padding=(0,2))
        self.conv2_bn = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(in_channels=22, out_channels=32, kernel_size=(1,3), stride=(1,1), padding=(0,1))
        self.conv3_bn = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(in_channels=22, out_channels=32, kernel_size=(1,1), stride=(1,1))
        self.conv4_bn = nn.BatchNorm2d(32)
        
        self.conv5 = nn.Conv2d(in_channels=160, out_channels=64, kernel_size=(1,9), stride=(1,1), padding=(0,4))
        self.conv5_bn = nn.BatchNorm2d(64)       
        self.conv6 = nn.Conv2d(in_channels=160, out_channels=64, kernel_size=(1,7), stride=(1,1), padding=(0,3))
        self.conv6_bn = nn.BatchNorm2d(64)
        self.conv7 = nn.Conv2d(in_channels=160, out_channels=64, kernel_size=(1,5), stride=(1,1), padding=(0,2))
        self.conv7_bn = nn.BatchNorm2d(64)
        self.conv8 = nn.Conv2d(in_channels=160, out_channels=64, kernel_size=(1,3), stride=(1,1), padding=(0,1))
        self.conv8_bn = nn.BatchNorm2d(64)        
        self.conv9 = nn.Conv2d(in_channels=160, out_channels=64, kernel_size=(1,1), stride=(1,1), padding=(0,0))
        self.conv9_bn = nn.BatchNorm2d(64)
        
        self.conv10 = nn.Conv2d(in_channels=320, out_channels=128, kernel_size=(1,9), stride=(1,1), padding=(0,4))
        self.conv10_bn = nn.BatchNorm2d(128)       
        self.conv11 = nn.Conv2d(in_channels=320, out_channels=128, kernel_size=(1,7), stride=(1,1), padding=(0,3))
        self.conv11_bn = nn.BatchNorm2d(128)
        self.conv12 = nn.Conv2d(in_channels=320, out_channels=128, kernel_size=(1,5), stride=(1,1), padding=(0,2))
        self.conv12_bn = nn.BatchNorm2d(128)
        self.conv13 = nn.Conv2d(in_channels=320, out_channels=128, kernel_size=(1,3), stride=(1,1), padding=(0,1))
        self.conv13_bn = nn.BatchNorm2d(128)        
        self.conv14 = nn.Conv2d(in_channels=320, out_channels=128, kernel_size=(1,1), stride=(1,1), padding=(0,0))
        self.conv14_bn = nn.BatchNorm2d(128)
        
        self.relu = nn.ELU(inplace=True) #nn.ReLU(inplace=True)
        
        self.fc = nn.Linear(in_features=640,  out_features=4)
        self.maxpool = nn.MaxPool2d(kernel_size=(1,10), stride=(1,5), padding=(0,0))
        self.avgpool = nn.AvgPool2d(kernel_size=(1,38), stride=(1,38), padding=(0,0))
        
        
        
    def forward(self, X):
        # Nx1x22x1000
        xi = X.reshape(X.size(0),1,X.size(1),X.size(2)).permute(0,2,1,3)
        x = self.conv0(xi)
        x = self.conv0_bn(x)
        x0 = self.relu(x)
        x = self.conv1(xi)
        x = self.conv1_bn(x)
        x1 = self.relu(x)
        x = self.conv2(xi)
        x = self.conv2_bn(x)
        x2 = self.relu(x)
        x = self.conv3(xi)
        x = self.conv3_bn(x)
        x3 = self.relu(x)
        x = self.conv4(xi)
        x = self.conv4_bn(x)
        x4 = self.relu(x)
        x = torch.cat((x0,x1,x2,x3,x4),1)
        
        xi = self.maxpool(x)
                
        x = self.conv5(xi)
        x = self.conv5_bn(x)
        x0 = self.relu(x)
        x = self.conv6(xi)
        x = self.conv6_bn(x)
        x1 = self.relu(x)
        x = self.conv7(xi)
        x = self.conv7_bn(x)
        x2 = self.relu(x)
        x = self.conv8(xi)
        x = self.conv8_bn(x)
        x3 = self.relu(x)
        x = self.conv9(xi)
        x = self.conv9_bn(x)
        x4 = self.relu(x)
        x = torch.cat((x0,x1,x2,x3,x4),1)
       
        xi = self.maxpool(x)

        x = self.conv10(xi)
        x = self.conv10_bn(x)
        x0 = self.relu(x)
        x = self.conv11(xi)
        x = self.conv11_bn(x)
        x1 = self.relu(x)
        x = self.conv12(xi)
        x = self.conv12_bn(x)
        x2 = self.relu(x)
        x = self.conv13(xi)
        x = self.conv13_bn(x)
        x3 = self.relu(x)
        x = self.conv14(xi)
        x = self.conv14_bn(x)
        x4 = self.relu(x)
        x = torch.cat((x0,x1,x2,x3,x4),1)

        x = self.avgpool(x)
        x = self.fc(x.squeeze())
        return x

In [0]:
# reduce maxpooling layers
class Net(Module):
    def __init__(self):
        super(Net, self).__init__()

        self.ave = nn.AvgPool1d(kernel_size = 10, stride = 10)
        # 22*100
        self.cnn_layers = Sequential(
            # first conv
            nn.Conv1d(in_channels = 22, out_channels = 32, kernel_size = 3, stride=1),
            nn.BatchNorm1d(32),
            nn.ELU(inplace=True),

            # second conv
            nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3, stride=1),
            nn.BatchNorm1d(32),
            nn.ELU(inplace=True),

            # third conv
            nn.Conv1d(in_channels = 32, out_channels = 32, kernel_size = 3, stride=1),
            nn.BatchNorm1d(32),
            nn.ELU(inplace=True),
            nn.MaxPool1d(kernel_size=4, stride=2),
            # 64*46
        )

        self.lstm= nn.LSTM(32, 16, 4, batch_first=True, dropout = 0.1, bidirectional = True)

        self.linear_layers = Sequential(
            Linear(46*32,128),
            nn.BatchNorm1d(128),
            ReLU(inplace=True),

            Linear(128,32),
            nn.BatchNorm1d(32),
            ReLU(inplace=True),

            Linear(32, 4)
        )
        
    def forward(self, x):
        x = self.ave(x)
        x = self.cnn_layers(x)
        x = x.transpose(1, 2)
        x,_ = self.lstm(x)
        x = x.reshape((-1, 46*32))
        x = self.linear_layers(x)
        return x
        

In [0]:
# class Net(Module):
#     def __init__(self):
#         super(Net, self).__init__()

#         self.ave = nn.AvgPool1d(kernel_size = 10, stride = 10)
#         # 22*50
#         self.fc1 = Linear(22, 10)
#         # 50*7
#         self.lstm= nn.LSTM(10, 64, 4, batch_first=True, dropout = 0)

#         # 64
#         self.linear_layers = Sequential(
#             Linear(64*100,128),
#             nn.BatchNorm1d(128),
#             ReLU(inplace=True),

#             Linear(128,32),
#             nn.BatchNorm1d(32),
#             ReLU(inplace=True),

#             Linear(32, 4)
#         )
        
#     def forward(self, x):
#         x = self.ave(x)
#         x = x.transpose(1, 2)
#         x = self.fc1(x)
#         x,_ = self.lstm(x)
#         # x = x[:, -1, :]
#         # x = self.linear_layers(x.squeeze())
#         x = x.reshape((-1, 64*100))
#         x = self.linear_layers(x)
#         return x
        

In [24]:
# defining the model
model = InceptionNet() #Net()
model = model.float()

# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

print(model)

InceptionNet(
  (conv0): Conv2d(22, 32, kernel_size=(1, 9), stride=(1, 1), padding=(0, 4))
  (conv0_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(22, 32, kernel_size=(1, 7), stride=(1, 1), padding=(0, 3))
  (conv1_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(22, 32, kernel_size=(1, 5), stride=(1, 1), padding=(0, 2))
  (conv2_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(22, 32, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
  (conv3_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(22, 32, kernel_size=(1, 1), stride=(1, 1))
  (conv4_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(160, 64, kernel_size=(1, 9), stride=(1, 1), padding=(0, 4))
  (conv5_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

In [25]:
model = InceptionNet()
model = model.float()
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
model.load_state_dict(torch.load("Inception_69_9.pth"))
## test accuracy
with torch.no_grad():
    output = model(X_test.to(device).cuda().float())
softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on test set
test_acc = accuracy_score(y_test.cpu(), predictions)
print(test_acc)

0.6997742663656885


In [0]:

##### hyperparameters#########
batch_size = 10
lrs = [1e-3]
weight_decay = 0 # regularization
optimizer = Adam(model.parameters(), lr=lrs[0],  weight_decay=weight_decay)
n_epochs = 200
train_losses = []
val_losses = []
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

for lr in lrs:
    optimizer = Adam(model.parameters(), lr=lr,  weight_decay=weight_decay)
    for epoch in range(n_epochs):
        train_loss = 0

        for i,data in enumerate(train_loader, 0):
            # extract data in this batch
            inputs, labels = data

            if torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()

            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            output = model(inputs.float())
            loss_train = criterion(output, labels.long())
            loss_train.backward()
            optimizer.step()

            #Print statistics
            train_loss += loss_train

        train_losses.append(train_loss)   
        ## training accuracy
        with torch.no_grad():
            output = model(X_train.cuda().float())
            
        softmax = torch.exp(output).cpu()
        prob = list(softmax.numpy())
        predictions = np.argmax(prob, axis=1)
        # accuracy on training set
        train_acc = accuracy_score(y_train.cpu(), predictions)

        # validation loss
        with torch.no_grad():
            output = model(X_val.cuda().float())
        val_loss = criterion(output, y_val.cuda().long())
        val_losses.append(val_loss)
        softmax = torch.exp(output).cpu()
        prob = list(softmax.numpy())
        predictions = np.argmax(prob, axis=1)
        # accuracy on validation set
        val_acc = accuracy_score(y_val.cpu(), predictions)

        ## test accuracy
        with torch.no_grad():
            output = model(X_test.to(device).cuda().float())
        softmax = torch.exp(output).cpu()
        prob = list(softmax.numpy())
        predictions = np.argmax(prob, axis=1)

        # accuracy on test set
        test_acc = accuracy_score(y_test.cpu(), predictions)

        print("Epoch {}, train_loss: {:.2f}, \t val_loss: {:.2f} ".format(
                    epoch+1, train_loss, val_loss))
        print("training accuracy = {:.2f}, \t validation accuracy = {:.2f}, \t test accuracy = {:.2f}".format(train_acc, val_acc, test_acc))